In [2]:
import numpy as np
import pandas as pd
import os
from pathlib import Path

from IPython.display import HTML
from tqdm import tqdm

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
out_dir_path = '../output/1/{}'

In [5]:
player_stats = pd.DataFrame()
cnt = 0
for game_id in tqdm(sorted(os.listdir(out_dir_path.format('')))):
    for play_id in sorted(os.listdir(out_dir_path.format(f'{game_id}'))):
        cnt += 1
        player_stats = player_stats.append(pd.read_pickle(out_dir_path.format(f'{game_id}/{play_id}/player_stats.pkl')))
        # with open(out_dir_path.format(f'{game_id}/{play_id}/{frame_id}'), 'rb') as f:
print(cnt)
player_stats = player_stats.loc[(player_stats.frameId>=14)&(player_stats.frameId<=47)]
player_stats.head()

100%|██████████| 13/13 [00:16<00:00,  1.29s/it]


774


,gameId,playId,frameId,frame_after_snap,nflId,displayName,team,team_pos,eppa_ind,ppc_trans
24,2018090600,1037,14,3,2507763.0,Mike Wallace,PHI,OFF,-0.055004,0.071870
25,2018090600,1037,14,3,2539334.0,Desmond Trufant,ATL,DEF,-0.023521,0.056737
26,2018090600,1037,14,3,2539653.0,Robert Alford,ATL,DEF,-0.008106,0.027377
27,2018090600,1037,14,3,2540158.0,Zach Ertz,PHI,OFF,-0.099357,0.090630
28,2018090600,1037,14,3,2543850.0,Ricardo Allen,ATL,DEF,0.000859,0.000320


In [11]:
player_stats.loc[player_stats.team_pos=='DEF'].sort_values('eppa_ind', ascending=False).head(10)

,gameId,playId,frameId,frame_after_snap,nflId,displayName,team,team_pos,eppa_ind,ppc_trans
388,2018090903,1515,37,26,2558065.0,Adoree' Jackson,TEN,DEF,0.758510,0.199200
373,2018090903,1515,36,25,2558065.0,Adoree' Jackson,TEN,DEF,0.749674,0.198360
325,2018090905,628,39,28,494287.0,Devin McCourty,NE,DEF,0.746985,0.186505
358,2018090903,1515,35,24,2558065.0,Adoree' Jackson,TEN,DEF,0.698198,0.186730
343,2018090903,1515,34,23,2558065.0,Adoree' Jackson,TEN,DEF,0.696559,0.190095
313,2018090905,628,38,27,494287.0,Devin McCourty,NE,DEF,0.679928,0.173009
328,2018090903,1515,33,22,2558065.0,Adoree' Jackson,TEN,DEF,0.673615,0.187704
313,2018090903,1515,32,21,2558065.0,Adoree' Jackson,TEN,DEF,0.634679,0.178694
352,2018090907,3977,39,28,2532848.0,Tashaun Gipson,JAX,DEF,0.629073,0.144254
301,2018090905,628,37,26,494287.0,Devin McCourty,NE,DEF,0.618644,0.161338


In [6]:
aggs = player_stats.groupby(['nflId', 'displayName', 'team', 'team_pos'], as_index=False)['eppa_ind', 'ppc_trans'].agg(['sum', 'mean', 'count']).reset_index()
aggs

nflId       displayName team team_pos   eppa_ind                  \
                                                      sum      mean count   
0        949.0  Jonathan Stewart  NYG      OFF  -1.340394 -0.016149    83   
1       1032.0      Jordy Nelson  OAK      OFF   2.291634  0.229163    10   
2       1302.0        Aqib Talib   LA      DEF   0.088189  0.008819    10   
3       1581.0    DeSean Jackson   TB      OFF  24.294702  0.127867   190   
4       2354.0   Wesley Woodyard  TEN      DEF  13.948414  0.029741   469   
..         ...               ...  ...      ...        ...       ...   ...   
474  2561155.0    Zaire Franklin  IND      DEF  -0.014787 -0.000308    48   
475  2561301.0       Chris Board  BAL      DEF   0.973706  0.008772   111   
476  2561328.0      Sharif Finch  TEN      DEF  -0.165515 -0.001478   112   
477  2561427.0     Vyncint Smith  HOU      OFF  51.630314  0.150088   344   
478  2561485.0    Janarion Grant  BAL      OFF  11.076108  0.240785    46   

     ppc_trans                  
           sum      mean count  
0    10.148626  0.122273    83  
1     2.551031  0.255103    10  
2     0.068499  0.006850    10  
3    24.675784  0.129873   190  
4    14.681959  0.031305   469  
..         ...       ...   ...  
474   2.396976  0.049937    48  
475   1.942762  0.017502   111  
476   3.450664  0.030809   112  
477  52.936483  0.153885   344  
478   6.677316  0.145159    46  

[479 rows x 10 columns]

In [7]:
aggs.loc[aggs.team_pos=='DEF'].sort_values(('eppa_ind', 'mean'), ascending=False).head(10)

nflId      displayName team team_pos   eppa_ind                  \
                                                     sum      mean count   
390  2558168.0    Rasul Douglas  PHI      DEF   3.434237  0.202014    17   
20    496733.0        Joe Haden  PIT      DEF  33.460111  0.133840   250   
82   2532920.0      Josh Norman  WAS      DEF  63.641211  0.125278   508   
273  2555178.0   Maurice Canady  BAL      DEF  13.437265  0.121056   111   
448  2560886.0     Greg Stroman  WAS      DEF  19.986110  0.120398   166   
147  2541162.0       A.J. Bouye  JAX      DEF  72.123355  0.119806   602   
281  2555277.0        Eli Apple  NYG      DEF  65.168903  0.119139   547   
460  2560963.0  Anthony Averett  BAL      DEF  21.071805  0.113289   186   
118  2539653.0    Robert Alford  ATL      DEF  74.903969  0.111797   670   
378  2558065.0  Adoree' Jackson  TEN      DEF  34.986787  0.107984   324   

     ppc_trans                  
           sum      mean count  
390   1.462410  0.086024    17  
20   16.848609  0.067394   250  
82   33.694045  0.066327   508  
273   8.479260  0.076390   111  
448   8.605199  0.051839   166  
147  38.685330  0.064261   602  
281  40.989480  0.074935   547  
460  13.428123  0.072194   186  
118  41.570898  0.062046   670  
378  21.676367  0.066902   324

In [6]:
aggs.loc[aggs.team_pos=='DEF'].sort_values(('ppc_trans', 'mean'), ascending=False).head(10)

nflId      displayName team team_pos   eppa_ind                  \
                                                     sum      mean count   
228  2558849.0   Cameron Sutton  PIT      DEF  40.498404  0.110350   367   
220  2558168.0    Rasul Douglas  PHI      DEF   3.655873  0.166176    22   
114  2550613.0   Malcolm Butler  TEN      DEF  59.064541  0.100450   588   
261  2560963.0  Anthony Averett  BAL      DEF  41.335922  0.141561   292   
149  2555178.0   Maurice Canady  BAL      DEF  18.321861  0.109712   167   
158  2555344.0      Artie Burns  PIT      DEF  62.129102  0.083060   748   
13    496733.0        Joe Haden  PIT      DEF  48.689234  0.127793   381   
179  2556367.0  William Jackson  CIN      DEF  83.951593  0.071266  1178   
64   2539653.0    Robert Alford  ATL      DEF  95.317072  0.102823   927   
237  2559053.0     Torry McTyer  MIA      DEF   5.324943  0.053787    99   

     ppc_trans                  
           sum      mean count  
228  32.093297  0.087448   367  
220   1.656237  0.075284    22  
114  42.967391  0.073074   588  
261  21.146067  0.072418   292  
149  11.512095  0.068935   167  
158  48.087549  0.064288   748  
13   24.010318  0.063019   381  
179  73.533601  0.062422  1178  
64   57.630340  0.062169   927  
237   5.979927  0.060403    99

In [7]:
aggs.loc[aggs.team_pos=='OFF'].sort_values(('eppa_ind', 'mean'), ascending=False).head(10)

nflId       displayName team team_pos    eppa_ind                  \
                                                       sum      mean count   
265  2560988.0  Antonio Callaway  CLE      OFF   25.687249  0.256872   100   
274  2561485.0    Janarion Grant  BAL      OFF   15.481952  0.245745    63   
126  2552424.0   Phillip Dorsett   NE      OFF  170.637310  0.218766   780   
197  2557870.0     Taywan Taylor  TEN      OFF   23.235176  0.203817   114   
22   2495453.0      Andre Holmes  BUF      OFF   36.681920  0.202663   181   
0       1032.0      Jordy Nelson  OAK      OFF    3.002385  0.200159    15   
105  2543847.0        John Brown  BAL      OFF  162.897367  0.190746   854   
246  2560729.0     Robert Foster  BUF      OFF   52.221044  0.184527   283   
192  2557034.0       Marvin Hall  ATL      OFF    8.701647  0.177585    49   
23   2495454.0       Julio Jones  ATL      OFF  123.082830  0.163892   751   

      ppc_trans                  
            sum      mean count  
265   16.419191  0.164192   100  
274    9.005012  0.142937    63  
126  122.241670  0.156720   780  
197   17.483014  0.153360   114  
22    24.607632  0.135954   181  
0      3.886203  0.259080    15  
105  124.267324  0.145512   854  
246   36.883458  0.130330   283  
192    9.363685  0.191096    49  
23   101.212500  0.134770   751

In [8]:
aggs.loc[aggs.team_pos=='OFF'].sort_values(('ppc_trans', 'mean'), ascending=False).head(10)

nflId            displayName team team_pos    eppa_ind            \
                                                            sum      mean   
0       1032.0           Jordy Nelson  OAK      OFF    3.002385  0.200159   
192  2557034.0            Marvin Hall  ATL      OFF    8.701647  0.177585   
265  2560988.0       Antonio Callaway  CLE      OFF   25.687249  0.256872   
126  2552424.0        Phillip Dorsett   NE      OFF  170.637310  0.218766   
68   2540145.0  Cordarrelle Patterson   NE      OFF   10.179731  0.161583   
197  2557870.0          Taywan Taylor  TEN      OFF   23.235176  0.203817   
256  2560920.0       James Washington  PIT      OFF    4.310219  0.038484   
105  2543847.0             John Brown  BAL      OFF  162.897367  0.190746   
273  2561427.0          Vyncint Smith  HOU      OFF   64.236438  0.132174   
274  2561485.0         Janarion Grant  BAL      OFF   15.481952  0.245745   

            ppc_trans                  
    count         sum      mean count  
0      15    3.886203  0.259080    15  
192    49    9.363685  0.191096    49  
265   100   16.419191  0.164192   100  
126   780  122.241670  0.156720   780  
68     63    9.705035  0.154048    63  
197   114   17.483014  0.153360   114  
256   112   16.636018  0.148536   112  
105   854  124.267324  0.145512   854  
273   486   70.379359  0.144813   486  
274    63    9.005012  0.142937    63

In [7]:
passes = pd.DataFrame()
for game_id in tqdm(sorted(os.listdir(out_dir_path.format('')))):
    for play_id in tqdm(sorted(os.listdir(out_dir_path.format(f'{game_id}')))):
        passes = passes.append(pd.read_pickle(out_dir_path.format(f'{game_id}/{play_id}/passes.pkl')))

100%|██████████| 13/13 [00:16<00:00,  1.25s/it]


In [10]:
passes.head()

,gameId,playId,frameId,frames_after_snap,off_team,def_team,eppa_tot,ppc_trans_tot,maxEPPA_x,maxEPPA_y,maxEPPA_T,maxEPPA_ppc_off,maxEPPA_ppc_def,maxEPPA_xyac,maxEPPA_xepa,maxEPPA_trans,maxEPPA_trans_denorm,maxEPPA_hist_trans,maxEPPA_hist_trans_denorm,maxEPPA_eppa,maxEPPA_eppa_denorm,maxEPPA_ppc_val,maxEPPA_ppc_trans,maxEPPA_ppc_trans_denorm,maxPPC_x,maxPPC_y,maxPPC_T,maxPPC_ppc_off,maxPPC_ppc_def,maxPPC_xyac,maxPPC_xepa,maxPPC_trans,maxPPC_trans_denorm,maxPPC_hist_trans,maxPPC_hist_trans_denorm,maxPPC_eppa,maxPPC_eppa_denorm,maxPPC_ppc_val,maxPPC_ppc_trans,maxPPC_ppc_trans_denorm,maxPPC_DEF_x,maxPPC_DEF_y,maxPPC_DEF_T,maxPPC_DEF_ppc_off,maxPPC_DEF_ppc_def,maxPPC_DEF_xyac,maxPPC_DEF_xepa,maxPPC_DEF_trans,maxPPC_DEF_trans_denorm,maxPPC_DEF_hist_trans,...,maxPPC_DEF_ppc_trans,maxPPC_DEF_ppc_trans_denorm,maxPPCbyVALUE_x,maxPPCbyVALUE_y,maxPPCbyVALUE_T,maxPPCbyVALUE_ppc_off,maxPPCbyVALUE_ppc_def,maxPPCbyVALUE_xyac,maxPPCbyVALUE_xepa,maxPPCbyVALUE_trans,maxPPCbyVALUE_trans_denorm,maxPPCbyVALUE_hist_trans,maxPPCbyVALUE_hist_trans_denorm,maxPPCbyVALUE_eppa,maxPPCbyVALUE_eppa_denorm,maxPPCbyVALUE_ppc_val,maxPPCbyVALUE_ppc_trans,maxPPCbyVALUE_ppc_trans_denorm,maxPPCbyTRANS_x,maxPPCbyTRANS_y,maxPPCbyTRANS_T,maxPPCbyTRANS_ppc_off,maxPPCbyTRANS_ppc_def,maxPPCbyTRANS_xyac,maxPPCbyTRANS_xepa,maxPPCbyTRANS_trans,maxPPCbyTRANS_trans_denorm,maxPPCbyTRANS_hist_trans,maxPPCbyTRANS_hist_trans_denorm,maxPPCbyTRANS_eppa,maxPPCbyTRANS_eppa_denorm,maxPPCbyTRANS_ppc_val,maxPPCbyTRANS_ppc_trans,maxPPCbyTRANS_ppc_trans_denorm,TRUEPASS_x,TRUEPASS_y,TRUEPASS_T,TRUEPASS_ppc_off,TRUEPASS_ppc_def,TRUEPASS_xyac,TRUEPASS_xepa,TRUEPASS_trans,TRUEPASS_trans_denorm,TRUEPASS_hist_trans,TRUEPASS_hist_trans_denorm,TRUEPASS_eppa,TRUEPASS_eppa_denorm,TRUEPASS_ppc_val,TRUEPASS_ppc_trans,TRUEPASS_ppc_trans_denorm
0,2018090600,1037,12,1,PHI,ATL,-0.545925,0.456155,53.5,46.5,0.8,0.670220,0.089919,10.294803,1.892704,0.001629,430.059061,0.000066,17.327636,0.002066,545.541972,1.268528,0.001092,288.234240,46.5,32.5,0.9,0.970593,0.018895,5.015805,-1.730154,0.000600,158.475784,0.000014,3.663910,-0.001008,-266.124448,-1.679275,0.000583,153.815450,79.5,27.5,2.6,0.000438,0.999562,7.658771,3.496741,5.460593e-11,0.000014,1.316298e-07,...,2.392518e-14,6.316248e-09,53.5,46.5,0.8,0.670220,0.089919,10.294803,1.892704,1.629012e-03,4.300591e+02,6.563499e-05,1.732764e+01,2.066447e-03,5.455420e+02,1.268528,1.091796e-03,2.882342e+02,51.5,39.5,0.7,0.890561,0.109439,1.170604,-1.611135,0.002442,644.736108,0.000064,16.959913,-0.003504,-925.076982,-1.434815,0.002175,574.177100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018090600,1037,13,2,PHI,ATL,-0.544500,0.452353,53.5,46.5,0.8,0.694787,0.086227,10.181234,1.892704,0.001726,455.637852,0.000066,17.327636,0.002270,599.175420,1.315026,0.001199,316.571191,46.5,32.5,0.9,0.971448,0.018732,5.460724,-1.730154,0.000603,159.285953,0.000014,3.663910,-0.001014,-267.720761,-1.680756,0.000586,154.738092,83.5,27.5,3.0,0.000398,0.999602,7.673952,3.735711,8.247259e-12,0.000002,2.290621e-08,...,3.279532e-15,8.657965e-10,53.5,46.5,0.8,0.694787,0.086227,10.181234,1.892704,1.725901e-03,4.556379e+02,6.563499e-05,1.732764e+01,2.269604e-03,5.991754e+02,1.315026,1.199133e-03,3.165712e+02,51.5,39.5,0.7,0.880699,0.119301,0.512420,-1.730154,0.002411,636.455124,0.000064,16.959913,-0.003673,-969.795130,-1.523745,0.002123,560.525257,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018090600,1037,14,3,PHI,ATL,-0.496992,0.447577,53.5,46.5,0.8,0.820822,0.067718,10.245103,1.892704,0.002190,578.074869,0.000066,17.327642,0.003402,898.081050,1.553572,0.001797,474.496413,46.5,32.5,0.9,0.971449,0.021129,4.309436,-1.735601,0.000596,157.378828,0.000014,3.663911,-0.001005,-265.348215,-1.686048,0.000579,152.885515,79.5,27.5,2.5,0.000362,0.999638,7.684849,3.496741,9.548892e-11,0.000025,3.085305e-07,...,3.460525e-14,9.135786e-09,53.5,46.5,0.8,0.820822,0.067718,10.245103,1.892704,2.189678e-03,5.780749e+02,6.563501e-05,1.732764e+01,3.401822e-03,8.980811e+

In [8]:
passes.TRUEPASS_xepa.describe()

count    774.000000
mean       0.865947
std        1.183073
min       -2.389349
25%        0.055750
50%        0.720112
75%        1.495628
max        4.947160
Name: TRUEPASS_xepa, dtype: float64

In [11]:
passes[['eppa_tot', 'ppc_trans_tot']].describe()

,eppa_tot,ppc_trans_tot
count,9816.000000,9816.000000
mean,0.388776,0.536429
std,0.401317,0.059889
min,-0.796343,0.370889
25%,0.098791,0.494433
50%,0.356851,0.536640
75%,0.642428,0.576597
max,2.075265,0.731022


In [12]:
passes.sort_values('eppa_tot', ascending=False)

,gameId,playId,frameId,frames_after_snap,off_team,def_team,eppa_tot,ppc_trans_tot,maxEPPA_x,maxEPPA_y,maxEPPA_T,maxEPPA_ppc_off,maxEPPA_ppc_def,maxEPPA_xyac,maxEPPA_xepa,maxEPPA_trans,maxEPPA_trans_denorm,maxEPPA_hist_trans,maxEPPA_hist_trans_denorm,maxEPPA_eppa,maxEPPA_eppa_denorm,maxEPPA_ppc_val,maxEPPA_ppc_trans,maxEPPA_ppc_trans_denorm,maxPPC_x,maxPPC_y,maxPPC_T,maxPPC_ppc_off,maxPPC_ppc_def,maxPPC_xyac,maxPPC_xepa,maxPPC_trans,maxPPC_trans_denorm,maxPPC_hist_trans,maxPPC_hist_trans_denorm,maxPPC_eppa,maxPPC_eppa_denorm,maxPPC_ppc_val,maxPPC_ppc_trans,maxPPC_ppc_trans_denorm,maxPPC_DEF_x,maxPPC_DEF_y,maxPPC_DEF_T,maxPPC_DEF_ppc_off,maxPPC_DEF_ppc_def,maxPPC_DEF_xyac,maxPPC_DEF_xepa,maxPPC_DEF_trans,maxPPC_DEF_trans_denorm,maxPPC_DEF_hist_trans,...,maxPPC_DEF_ppc_trans,maxPPC_DEF_ppc_trans_denorm,maxPPCbyVALUE_x,maxPPCbyVALUE_y,maxPPCbyVALUE_T,maxPPCbyVALUE_ppc_off,maxPPCbyVALUE_ppc_def,maxPPCbyVALUE_xyac,maxPPCbyVALUE_xepa,maxPPCbyVALUE_trans,maxPPCbyVALUE_trans_denorm,maxPPCbyVALUE_hist_trans,maxPPCbyVALUE_hist_trans_denorm,maxPPCbyVALUE_eppa,maxPPCbyVALUE_eppa_denorm,maxPPCbyVALUE_ppc_val,maxPPCbyVALUE_ppc_trans,maxPPCbyVALUE_ppc_trans_denorm,maxPPCbyTRANS_x,maxPPCbyTRANS_y,maxPPCbyTRANS_T,maxPPCbyTRANS_ppc_off,maxPPCbyTRANS_ppc_def,maxPPCbyTRANS_xyac,maxPPCbyTRANS_xepa,maxPPCbyTRANS_trans,maxPPCbyTRANS_trans_denorm,maxPPCbyTRANS_hist_trans,maxPPCbyTRANS_hist_trans_denorm,maxPPCbyTRANS_eppa,maxPPCbyTRANS_eppa_denorm,maxPPCbyTRANS_ppc_val,maxPPCbyTRANS_ppc_trans,maxPPCbyTRANS_ppc_trans_denorm,TRUEPASS_x,TRUEPASS_y,TRUEPASS_T,TRUEPASS_ppc_off,TRUEPASS_ppc_def,TRUEPASS_xyac,TRUEPASS_xepa,TRUEPASS_trans,TRUEPASS_trans_denorm,TRUEPASS_hist_trans,TRUEPASS_hist_trans_denorm,TRUEPASS_eppa,TRUEPASS_eppa_denorm,TRUEPASS_ppc_val,TRUEPASS_ppc_trans,TRUEPASS_ppc_trans_denorm
18,2018090903,685,30,19,TEN,MIA,2.075265,0.548812,112.5,23.5,0.7,0.846433,0.052917,4.737433,4.256342,0.001313,346.609218,0.000170,45.007992,0.004730,1248.731279,3.602706,0.001111,293.381310,116.5,26.5,1.4,0.963879,0.036121,10.959315,4.256342,0.000012,3.058689,1.238043e-06,0.326843,0.000048,12.548575,4.102599,0.000011,2.948206,107.5,14.5,1.1,0.003018,0.996982,2.206473,4.256342,3.864475e-08,1.020221e-02,2.356726e-05,...,1.166371e-10,3.079220e-05,116.5,26.5,1.4,0.963879,0.036121,10.959315,4.256342,1.158594e-05,3.058689,1.238043e-06,0.326843,4.753248e-05,12.548575,4.102599,1.116745e-05,2.948206,112.5,23.5,0.7,0.846433,0.052917,4.737433,4.256342,0.001313,346.609218,0.000170,45.007992,0.004730,1248.731279,3.602706,0.001111,293.381310,107.5,8.5,1.0,0.681167,0.318833,2.401562,4.256342,0.000755,199.365091,0.000136,35.85971,0.002189,578.01518,2.89928,0.000514,135.800915
17,2018090903,685,29,18,TEN,MIA,2.056661,0.554899,112.5,23.5,0.7,0.870614,0.109115,3.836976,4.256342,0.001287,339.808950,0.000168,44.233422,0.004770,1259.206055,3.705629,0.001121,295.842291,118.5,26.5,1.5,0.940557,0.059443,5.428627,4.256342,0.000005,1.354884,5.949418e-07,0.157065,0.000021,5.424051,4.003332,0.000005,1.274346,106.5,15.5,1.1,0.004632,0.995368,-1.533758,-2.285677,5.087423e-08,1.343080e-02,1.706407e-05,...,2.356553e-10,6.221301e-05,118.5,26.5,1.5,0.940557,0.059443,5.428627,4.256342,5.132137e-06,1.354884,5.949418e-07,0.157065,2.054565e-05,5.424051,4.003332,4.827067e-06,1.274346,112.5,23.5,0.7,0.870614,0.109115,3.836976,4.256342,0.001287,339.808950,0.000168,44.233422,0.004770,1259.206055,3.705629,0.001121,295.842291,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,2018090903,685,28,17,TEN,MIA,2.022709,0.548386,112.5,24.5,0.7,0.804601,0.093760,4.164648,4.256342,0.001186,313.109302,0.000168,44.233409,0.004062,1072.292064,3.424657,0.000954,251.928062,116.5,28.5,1.4,0.951667,0.048333,14.571370,4.256342,0.000006,1.686834,7.017237e-07,0.185255,0.000026,6.832722,4.050620,0.000006,1.605304,105.5,17.5,1.1,0.005979,0.994021,0.991216,-2.285677,4.331927e-08,1.143629e-02,9.552783e-06,...,2.590194e-10,6.838111e-05,116.5,28.5,1.4,0.951667,0.048333,14.571370,4.256342,6.389521e-06,1.686834,7.0172